In [1]:
import os
from tryRAG.framework import RAGFramework

/home/yasaisen/anaconda3/envs/MVLM_RAG/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
doc_path = os.path.join(os.getcwd(), "../..", "ref_docs", "eecs_20250606_text_bs_rewritten.jsonl")
idx_path = os.path.join(os.getcwd(), "..", "ref_idx", "paragraph_p")

cfg = {
    "lm_model_name": "../../gemma-3-4b-it", 
    "emb_model_name": "all-MiniLM-L6-v2", 
    "mode": "hybrid", #@ "hybrid" / "sparse" / "hybrid"
    "chunk_level": "paragraph", #@ "web_page" / "paragraph" / "sentence"
    "more_info": True, #@ True / False
    # "doc_path": doc_path, 
    "idx_path": idx_path, 
    "device": "cuda", 
}
USE_UPPER_TEXT = False
USE_PRE_ANSWER = False
TOP_K = 5

rag = RAGFramework.from_config(cfg)

Loading checkpoint shards: 100%|██████████| 2/2 [00:02<00:00,  1.00s/it]


Loaded ['dense', 'sparse'] index from /home/yasaisen/Desktop/29_research/research_main/lab_05/to_git/tryRAG/../ref_idx/paragraph_p
RAGFramework initialized



In [3]:
question = 'For how long has Berkeley EECS been at the forefront of research? Available options:\n    (A) Less than a decade\n    (B) Over a quarter of a century\n    (C) Over half a century\n    (D) Over a century'

response = rag.ask(question, top_k=TOP_K)
print(response['response'])

print('=' * 20)
for doc in response['relevant_docs']:
    print(doc.idx, doc.url)

Answer: (C) Over half a century. The text states that Berkeley EECS has been at the forefront of research for “over half a century.”
13572 https://eecs.berkeley.edu/research/colloquium/archives/spring2021
17551 https://eecs.berkeley.edu/category/research/page/4
58708 https://eecs.berkeley.edu/category/research/page/76
17550 https://eecs.berkeley.edu/category/research/page/4
44627 https://eecs.berkeley.edu/category/research/page/12
18856 http://www.eecs.berkeley.edu/IPRO/BEARS/2011
18855 http://www.eecs.berkeley.edu/IPRO/BEARS/2011
18881 http://www.eecs.berkeley.edu/IPRO/BEARS/2010
18882 http://www.eecs.berkeley.edu/IPRO/BEARS/2010
19411 http://www.eecs.berkeley.edu/IPRO/BEARS/open-house/12/trust.html


In [4]:
question = 'Who is Lee Julian Purnell'

response = rag.ask(question, top_k=TOP_K)
print(response['response'])

print('=' * 20)
for doc in response['relevant_docs']:
    print(doc.idx, doc.url)

Lee Julian Purnell is the first Black student known to have graduated from the EECS department at UC Berkeley. He was born in Washington, D.C. in 1896, graduated from Berkeley High in 1915, earned a B.A. from Cal in 1919, and later obtained a B.S. in Electrical Engineering from MIT in 1921 and an M.S. in Electrical Engineering from Berkeley in 19
63613 https://eecs.berkeley.edu/category/people/page/29
63461 https://eecs.berkeley.edu/news/page/27/?field_eecs_news_topics_target_id_entityreference_filter=61
63279 https://eecs.berkeley.edu/news/page/27/?field_eecs_news_topics_target_id_entityreference_filter=68
14766 https://eecs.berkeley.edu/blog/page/7
63462 https://eecs.berkeley.edu/news/page/27/?field_eecs_news_topics_target_id_entityreference_filter=61
63154 https://eecs.berkeley.edu/news/page/27/?field_eecs_news_topics_target_id_entityreference_filter=60
63023 https://eecs.berkeley.edu/news/page/27/?field_eecs_news_topics_target_id_entityreference_filter=64
65052 https://eecs.berkele

In [5]:
from tryRAG.evaluate import Evaluator, batchTestRunner

In [9]:
testrunner = batchTestRunner(
    rag=rag, 
    dataset_type='optionalQA', #@ 'factoidQA' / 'optionalQA'
)

14241 dict_keys(['chunk_url', 'content', 'question', 'answer'])


In [10]:
result_dict = testrunner.test(
    TOP_K=TOP_K, 
    USE_UPPER_TEXT=USE_UPPER_TEXT, 
    USE_PRE_ANSWER=USE_PRE_ANSWER,
)

100%|██████████| 200/200 [04:35<00:00,  1.38s/it]


In [8]:
evaluator = Evaluator()
eval_res = evaluator.evaluate(
    result_dict=result_dict,
)
eval_res

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


{'bleu': 0.06862745098039216,
 'brevity_penalty': 0.947220561907903,
 'length_ratio': 7.748697054579407,
 'translation_length': 12.980392156862745,
 'reference_length': 2.392156862745098,
 'rouge1': 0.49146477989188986,
 'rouge2': 0.3238426949727259,
 'rougeL': 0.49146477989188986,
 'rougeLsum': 0.49146477989188986,
 'hit_rate': 0.4411764705882353,
 'backHit_rate': 0.32435807656395893}